# Evolving surfaces

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
import time
from scipy.interpolate import interp1d

In [ ]:
starry.config.lazy = False
starry.config.quiet = True
np.random.seed(0)

## Generate the dataset

### Process parameters

In [ ]:
# Ylm parameters
ydeg_max = 20
ydeg = 15

# Geometry
inc = 75.0

# Light curve
tmax = 200.0
npts = 10000
ferr = 1e-4

# Rotation
prot = 3.1
alpha = 0.025

# Spots
tau_mu = 60.0
tau_sd = 5.0
tau = lambda: tau_mu + tau_sd * np.random.randn()  # spot duration in days
spotfreq = 0.25  # avg number of new spots / day

# Spot latitude distribution
lat = lambda: (np.arccos(2 * np.random.random() - 1) - 0.5 * np.pi) * 180 / np.pi
# lat = lambda: (-1 if np.random.random() < 0.5 else 1)* 30.0 + 10.0 * np.random.randn()
# lat = lambda: 15.0 * np.random.randn()

# Spot longitude distribution: isotropic
lon = lambda: 360.0 * np.random.random()

# Spot size distribution
sigma = lambda: max(0.01, np.exp(-3.5 + 0.4 * np.random.randn()))

# Spot intensity distribution
intensity = lambda: -min(0.5, np.exp(-3 + 0.5 * np.random.randn()))

# Spot amplitude as a function of time
def amplitude(time, t0, amp=0.1, b=25, dur=0.45):
    phase = (time - t0) / tau()
    t1 = 0.5 * (1 - dur)
    t2 = 0.5 * (1 + dur)
    f1 = np.exp(b * (phase - t1)) / (np.exp(b * (phase - t1)) + 1)
    f2 = 1 - np.exp(b * (phase - t2)) / (np.exp(b * (phase - t2)) + 1)
    f = np.minimum(f1, f2)
    f0 = np.exp(-t1 * b) / (np.exp(-t1 * b) + 1)
    f -= f0
    f[phase < 0] = 0.0
    f[phase > 1] = 0.0
    return f

In [ ]:
# Spot size distribution
sigma = lambda: max(0.01, np.exp(-3.5 + 0.4 * np.random.randn()))
plt.hist([sigma() for k in range(10000)], bins=30)
plt.xlabel("spot size");

In [ ]:
# Spot intensity distribution
plt.hist([np.abs(intensity()) for k in range(10000)], bins=30)
plt.xlabel("|intensity|");

In [ ]:
# Spot latitude distribution
plt.hist([lat() for k in range(10000)], bins=30)
plt.xlim(-90, 90)
plt.xlabel("latitude");

In [ ]:
# Time array
time = np.linspace(0, tmax, npts)

In [ ]:
# Sample amplitude as a function of time
plt.plot(time, amplitude(time, 0));

In [ ]:
# Spot emergence times
x = np.random.random(npts)
dt = time[1] - time[0]
t0 = np.sort((1 + tau_mu / tmax) * time[x < dt * spotfreq] - tau_mu)
nspots = len(t0)
print(nspots)

In [ ]:
# Spot amplitudes
amp = np.array([amplitude(time, t0[k]) for k in range(nspots)])

In [ ]:
plt.figure(figsize=(12, 8))
plt.imshow(amp, aspect="auto", extent=(0, tmax, nspots, 0), vmin=0, vmax=1)
plt.colorbar()
plt.xlabel("time [days]")
plt.ylabel("spot number");

### Spot components

In [ ]:
# Setup figure
nx = 1 + int(np.ceil(np.sqrt(nspots)))
ny = 1
while ny * nx < nspots:
    ny += 1
fig, ax = plt.subplots(ny, nx, figsize=(12, 8))
ax = ax.flatten()
for axis in ax:
    axis.axis("off")

# Generate maps
map = starry.Map(ydeg_max)
y = np.zeros((nspots, (ydeg_max + 1) ** 2))
for k in tqdm(range(nspots)):
    map.reset()
    map.add_spot(
        lat=lat(), lon=lon(), sigma=sigma(), intensity=intensity(), relative=False
    )
    map[ydeg + 1 :, :] = 0.0
    y[k] = map.amp * map.y
    img = np.pi * map.render(projection="moll", res=100)
    ax[k].imshow(
        img, origin="lower", extent=(-1, 1, -0.5, 0.5), cmap="Greys_r", vmax=1,
    )
    x_el = np.linspace(-1, 1, 1000)
    y_el = 0.5 * np.sqrt(1 - x_el ** 2)
    ax[k].plot(x_el, y_el, "k-", lw=1, clip_on=False)
    ax[k].plot(x_el, -y_el, "k-", lw=1, clip_on=False)

### Compute the flux

In [ ]:
map = starry.Map(ydeg_max)
map.inc = inc

# No diff rot
map.alpha = 0.0
flux_no_diff_rot = np.ones_like(time)
for k in tqdm(range(nspots)):
    theta = 360.0 / prot * (time - t0[k])
    map[:, :] = y[k]
    flux_no_diff_rot += amp[k] * (map.flux(theta=theta) - 1.0)

# With diff rot
map.alpha = alpha
flux0 = np.ones_like(time)
for k in tqdm(range(nspots)):
    theta = 360.0 / prot * (time - t0[k])
    map[:, :] = y[k]
    flux0 += amp[k] * (map.flux(theta=theta) - 1.0)

In [ ]:
plt.plot(time, flux_no_diff_rot, lw=1, color="C1", alpha=0.5)
plt.plot(time, flux0, "C0-");

In [ ]:
np.savez(
    "lightcurve.npz",
    time=time,
    flux0=flux0,
    ydeg=ydeg,
    inc=inc,
    alpha=alpha,
    y=y,
    prot=prot,
    ydeg_max=ydeg_max,
)

### Image

In [ ]:
res = 300
downsamp = 100
nim = len(time[::downsamp])
img = np.ones((nim, res, res))
map.alpha = alpha
for k in tqdm(range(nspots)):
    theta = 360.0 / prot * (time - t0[k])
    map[:, :] = y[k]
    imgk = np.pi * map.render(projection="moll", res=res, theta=theta[::downsamp]) - 1.0
    img += amp[k, ::downsamp].reshape(-1, 1, 1) * imgk
map.show(image=img, projection="moll", colorbar=True)

In [ ]:
map.alpha = 0.1
X = map.design_matrix(theta=theta)
plt.imshow(np.log10(np.abs(X)), aspect="auto")

In [ ]:
map.alpha = 0.1
X = map.design_matrix(theta=theta - 30 * 360)
plt.imshow(np.log10(np.abs(X)), aspect="auto")